# Movie Analysis Tutorial

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/03-movie-analysis.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/03-movie-analysis.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/03-movie-analysis.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table>

### Start EVA server

We are reusing the start server notebook for launching the EVA server.

In [1]:
!wget -nc "https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/00-start-eva-server.ipynb"
%run 00-start-eva-server.ipynb
cursor = connect_to_server()

File '00-start-eva-server.ipynb' already there; not retrieving.

kill -9 $(lsof -ti:5432)


sh: 1: kill: Usage: kill [-s sigspec | -signum | -sigspec] [pid | job]... or
kill -l [exitstatus]


nohup eva_server > eva.log 2>&1 &

Note: you may need to restart the kernel to use updated packages.


# Download custom UDFs, model, and video

In [2]:
# Download UDF
!wget -nc https://www.dropbox.com/s/lharq14izp08bfz/gender.py

# Download built-in Face Detector
!wget -nc https://raw.githubusercontent.com/georgia-tech-db/eva/master/eva/udfs/face_detector.py

# Download models
!wget -nc https://www.dropbox.com/s/0y291evpqdfmv2z/gender.pth

# Download videos
!wget -nc https://www.dropbox.com/s/f5447euuuis1vdy/short.mp4

File 'gender.py' already there; not retrieving.

File 'gender.pth' already there; not retrieving.

File 'short.mp4' already there; not retrieving.



### Load video for analysis

In [3]:
cursor.execute("DROP TABLE TIKTOK;")
response = cursor.fetch_all()
print(response)
cursor.execute("LOAD FILE 'short.mp4' INTO TIKTOK;")
response = cursor.fetch_all()
print(response)
cursor.execute("""SELECT id FROM TIKTOK WHERE id < 5""")
response = cursor.fetch_all()
print(response)

@status: 0
@batch: Batch Object:
@dataframe:                                     0
0  Table Successfully dropped: TIKTOK
@batch_size: 1
@identifier_column: None
@query_time: 0.04610173869878054
@status: 0
@batch: Batch Object:
@dataframe:                                                  0
0  Video successfully added at location: short.mp4
@batch_size: 1
@identifier_column: None
@query_time: 0.05747787281870842
@status: 0
@batch: Batch Object:
@dataframe:    tiktok.id
0          0
1          1
2          2
3          3
4          4
@batch_size: 5
@identifier_column: None
@query_time: 0.2248841244727373


### Visualize Video

In [4]:
from IPython.display import Video
Video("short.mp4", embed=True)

### Create GenderCNN and FaceDetector UDFs

In [11]:
cursor.execute("""DROP UDF GenderCNN;""")
response = cursor.fetch_all()
print(response)

cursor.execute("""CREATE UDF IF NOT EXISTS 
                  GenderCNN
                  INPUT (data NDARRAY UINT8(3, 224, 224)) 
                  OUTPUT (label TEXT(10)) 
                  TYPE  Classification 
                  IMPL 'gender.py';
        """)
response = cursor.fetch_all()
print(response)

cursor.execute("""CREATE UDF IF NOT EXISTS
                  FaceDetector
                  INPUT  (frame NDARRAY UINT8(3, ANYDIM, ANYDIM))
                  OUTPUT (bboxes NDARRAY FLOAT32(ANYDIM, 4),
                          scores NDARRAY FLOAT32(ANYDIM))
                  TYPE  FaceDetection
                  IMPL  'face_detector.py';
        """)
response = cursor.fetch_all()
print(response)

@status: -1
@batch: None
@error: UDF GenderCNN does not exist, therefore cannot be dropped.
@status: 0
@batch: Batch Object:
@dataframe:                                                    0
0  UDF GenderCNN successfully added to the database.
@batch_size: 1
@identifier_column: None
@query_time: 2.2618190944194794
@status: -1
@batch: None
@error: UDF FaceDetector already exists.


### Run Face Detector on video

In [12]:
cursor.execute("""SELECT id, FaceDetector(data).bboxes 
                  FROM TIKTOK WHERE id < 10""")
response = cursor.fetch_all()
print(response)

@status: 0
@batch: Batch Object:
@dataframe:    tiktok.id                                facedetector.bboxes
0          0  [[90.7062988281, 208.44972229, 281.6464233398,...
1          1  [[91.0182113647, 208.2758789062, 281.080810546...
2          2  [[90.3585586548, 207.3743286133, 283.439971923...
3          3  [[90.6942672729, 207.5603027344, 284.378173828...
4          4  [[90.6849746704, 208.9865722656, 282.128082275...
5          5  [[89.4743041992, 209.3809204102, 283.459381103...
6          6  [[88.500831604, 208.3154296875, 283.2917175293...
7          7  [[89.8386306763, 206.076171875, 282.9394226074...
8          8  [[90.1851730347, 224.3558044434, 281.297302246...
9          9  [[94.3444137573, 234.1324768066, 279.647613525...
@batch_size: 10
@identifier_column: None
@query_time: 0.39044602680951357


# Composing UDFs in a query

Detect gender of the faces detected in the video by composing a set of UDFs (GenderCNN, FaceDetector, and Crop)

In [13]:
cursor.execute("""SELECT id, bbox, GenderCNN(Crop(data, bbox)) 
                  FROM TIKTOK JOIN LATERAL  UNNEST(FaceDetector(data)) AS Face(bbox, conf)  
                  WHERE id < 10;""")
response = cursor.fetch_all()
print(response)

@status: 0
@batch: Batch Object:
@dataframe:    tiktok.id                                          Face.bbox  \
0          0  [90.7062988281, 208.44972229, 281.6464233398, ...   
1          1  [91.0182113647, 208.2758789062, 281.0808105469...   
2          2  [90.3585586548, 207.3743286133, 283.4399719238...   
3          3  [90.6942672729, 207.5603027344, 284.3781738281...   
4          4  [90.6849746704, 208.9865722656, 282.1280822754...   
5          5  [89.4743041992, 209.3809204102, 283.4593811035...   
6          6  [88.500831604, 208.3154296875, 283.2917175293,...   
7          7  [89.8386306763, 206.076171875, 282.9394226074,...   
8          8  [90.1851730347, 224.3558044434, 281.2973022461...   
9          9  [94.3444137573, 234.1324768066, 279.6476135254...   

  gendercnn.label  
0          female  
1          female  
2          female  
3          female  
4          female  
5          female  
6          female  
7          female  
8          female  
9          female

: 

### Visualize Output

In [8]:
import cv2
from matplotlib import pyplot as plt

def annotate_video(detections, input_video_path, output_video_path):
    color=(207, 248, 64)
    thickness=4

    vcap = cv2.VideoCapture(input_video_path)
    width = int(vcap.get(3))
    height = int(vcap.get(4))
    fps = vcap.get(5)
    fourcc = cv2.VideoWriter_fourcc(*'MP4V') #codec
    video=cv2.VideoWriter(output_video_path, fourcc, fps, (width,height))

    frame_id = 0
    # Capture frame-by-frame
    ret, frame = vcap.read()  # ret = 1 if the video is captured; frame is the image

    while ret:
        df = detections
        df = df[['Face.bbox', 'gendercnn.label']][df['tiktok.id'] == frame_id]
        
        if df.size:
            for bbox, label in df.values:
                x1, y1, x2, y2 = bbox
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                img=cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness) # object bbox
                cv2.putText(img, str(label), (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, thickness-1) # object label
            video.write(img)

        # Show every fifth frame
        if frame_id % 5 == 0:
            plt.imshow(img)
            plt.show()

        if frame_id == 20:
            return

        frame_id+=1
        ret, frame = vcap.read()

    video.release()
    vcap.release()

In [9]:
#!pip install ipywidgets
from ipywidgets import Video
input_path = 'short.mp4'
output_path = 'annotated_short.mp4'

dataframe = response.batch.frames
annotate_video(dataframe, input_path, output_path)

AttributeError: 'NoneType' object has no attribute 'frames'